In [1]:
# Tutorial source: https://nbviewer.jupyter.org/github/krasserm/face-recognition/blob/master/face-recognition.ipynb?flush_cache=true

In [13]:
from keras.models import load_model
from keras.utils import CustomObjectScope

import tensorflow as tf

In [14]:
from model import create_model


In [15]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

OSError: Unable to open file (Unable to open file: name = 'weights/nn4.small2.v1.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)